List of operations:
* [Joins](#joins)
* [Drop Duplicates](#drop_duplicates)
* [Group By](#group_by)
* [Order By](#order_by)
* [Filter using where](#filter_using_where)
* [Filter using like](#filter_using_like)
* [Count distinct values in a column](#count_distinct_values_in_a_column)
* [Count null rows per column](#count_null_rows_per_column)
* [Get items of a column in table A not present in a column of table B](#get_items_A_not_in_B)
* [Get the items not common to both column in table A and column in table B](#get_items_not_in_A_B)
<!-- * [Joins](#joins) -->

# Import Library

In [1]:
from pyspark.sql.types import *
from handyspark import *
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np

In [2]:
#####################
# Create SparkSession
#####################
sc, sql_sc = [None, None]
try:
    sc = SparkSession \
        .builder \
        .appName("spark-flo") \
        .config("spark.executor.memory", "30g") \
        .config("spark.driver.memory", "30g") \
        .config("spark.driver.allowMultipleContexts", "false") \
        .enableHiveSupport() \
        .getOrCreate()

    sql_sc = SQLContext(sc)
except SparkSessionError:
    print("Spark Session Failed to initialize.")
    pass

# Create spark dataframe from pandas dataframe

In [3]:
pdf1 = pd.DataFrame({'name':['Doraemon','Hachi','Shinchan','Felix','Felix']\
                     ,'score':[24,68,21,49,49]\
                     ,'kind':['Cat','Bee','Human','Cat','Cat']
                     ,'enrollment_date':['2019-01-02','2018-12-08','2018-06-05','2018-11-12','2018-11-12']})
pdf2 = pd.DataFrame({'name':['Nobita','Doraemon','Dorami','Ratatouile']\
                     ,'score':[60,21,45,30]\
                     ,'kind':['Human','Cat','Cat','Mouse']
                     ,'enrollment_date':['2019-12-02','2018-11-01','2018-06-15','2018-10-12']})


In [4]:
pdf1

,name,score,kind,enrollment_date
0,Doraemon,24,Cat,2019-01-02
1,Hachi,68,Bee,2018-12-08
2,Shinchan,21,Human,2018-06-05
3,Felix,49,Cat,2018-11-12
4,Felix,49,Cat,2018-11-12


In [5]:
pdf2

,name,score,kind,enrollment_date
0,Nobita,60,Human,2019-12-02
1,Doraemon,21,Cat,2018-11-01
2,Dorami,45,Cat,2018-06-15
3,Ratatouile,30,Mouse,2018-10-12


In [6]:
sdf1 = sc.createDataFrame(pdf1)
sdf2 = sc.createDataFrame(pdf2)

In [7]:
sdf1.show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Doraemon|   24|  Cat|     2019-01-02|
|   Hachi|   68|  Bee|     2018-12-08|
|Shinchan|   21|Human|     2018-06-05|
|   Felix|   49|  Cat|     2018-11-12|
|   Felix|   49|  Cat|     2018-11-12|
+--------+-----+-----+---------------+



In [8]:
sdf2.show()

+----------+-----+-----+---------------+
|      name|score| kind|enrollment_date|
+----------+-----+-----+---------------+
|    Nobita|   60|Human|     2019-12-02|
|  Doraemon|   21|  Cat|     2018-11-01|
|    Dorami|   45|  Cat|     2018-06-15|
|Ratatouile|   30|Mouse|     2018-10-12|
+----------+-----+-----+---------------+



# Joins <a id='joins'></a>

### pandas

In [9]:
pdf1.set_index('name').join(pdf2.set_index('name'),how='left',lsuffix='_pdf1').reset_index()

,name,score_pdf1,kind_pdf1,enrollment_date_pdf1,score,kind,enrollment_date
0,Doraemon,24,Cat,2019-01-02,21.0,Cat,2018-11-01
1,Felix,49,Cat,2018-11-12,NaN,NaN,NaN
2,Felix,49,Cat,2018-11-12,NaN,NaN,NaN
3,Hachi,68,Bee,2018-12-08,NaN,NaN,NaN
4,Shinchan,21,Human,2018-06-05,NaN,NaN,NaN


In [10]:
pdf1.merge(pdf2,on='name',how='left',suffixes=['_pdf1','_pdf2'])

,name,score_pdf1,kind_pdf1,enrollment_date_pdf1,score_pdf2,kind_pdf2,enrollment_date_pdf2
0,Doraemon,24,Cat,2019-01-02,21.0,Cat,2018-11-01
1,Hachi,68,Bee,2018-12-08,NaN,NaN,NaN
2,Shinchan,21,Human,2018-06-05,NaN,NaN,NaN
3,Felix,49,Cat,2018-11-12,NaN,NaN,NaN
4,Felix,49,Cat,2018-11-12,NaN,NaN,NaN


### sql

In [11]:
sql_sc.registerDataFrameAsTable(sdf1,'table1')
sql_sc.registerDataFrameAsTable(sdf2,'table2')

In [12]:
sql_sc.sql("""    
select *
from table1
left join table2 on table1.name = table2.name
""").collect()

[Row(name='Felix', score=49, kind='Cat', enrollment_date='2018-11-12', name=None, score=None, kind=None, enrollment_date=None),
 Row(name='Felix', score=49, kind='Cat', enrollment_date='2018-11-12', name=None, score=None, kind=None, enrollment_date=None),
 Row(name='Shinchan', score=21, kind='Human', enrollment_date='2018-06-05', name=None, score=None, kind=None, enrollment_date=None),
 Row(name='Doraemon', score=24, kind='Cat', enrollment_date='2019-01-02', name='Doraemon', score=21, kind='Cat', enrollment_date='2018-11-01'),
 Row(name='Hachi', score=68, kind='Bee', enrollment_date='2018-12-08', name=None, score=None, kind=None, enrollment_date=None)]

In [13]:
sql_sc.sql("""    
select *
from table1
left join table2 on table1.name = table2.name
""").show()

+--------+-----+-----+---------------+--------+-----+----+---------------+
|    name|score| kind|enrollment_date|    name|score|kind|enrollment_date|
+--------+-----+-----+---------------+--------+-----+----+---------------+
|   Felix|   49|  Cat|     2018-11-12|    null| null|null|           null|
|   Felix|   49|  Cat|     2018-11-12|    null| null|null|           null|
|Shinchan|   21|Human|     2018-06-05|    null| null|null|           null|
|Doraemon|   24|  Cat|     2019-01-02|Doraemon|   21| Cat|     2018-11-01|
|   Hachi|   68|  Bee|     2018-12-08|    null| null|null|           null|
+--------+-----+-----+---------------+--------+-----+----+---------------+



### spark

In [14]:
t1 = sdf1.alias('t1_alias')
t2 = sdf2.alias('t2_alias')

In [15]:
t1.join(t2, t1.name == t2.name, how='left').collect()

[Row(name='Felix', score=49, kind='Cat', enrollment_date='2018-11-12', name=None, score=None, kind=None, enrollment_date=None),
 Row(name='Felix', score=49, kind='Cat', enrollment_date='2018-11-12', name=None, score=None, kind=None, enrollment_date=None),
 Row(name='Shinchan', score=21, kind='Human', enrollment_date='2018-06-05', name=None, score=None, kind=None, enrollment_date=None),
 Row(name='Doraemon', score=24, kind='Cat', enrollment_date='2019-01-02', name='Doraemon', score=21, kind='Cat', enrollment_date='2018-11-01'),
 Row(name='Hachi', score=68, kind='Bee', enrollment_date='2018-12-08', name=None, score=None, kind=None, enrollment_date=None)]

In [16]:
t1.join(t2, t1.name == t2.name, how='left').show()

+--------+-----+-----+---------------+--------+-----+----+---------------+
|    name|score| kind|enrollment_date|    name|score|kind|enrollment_date|
+--------+-----+-----+---------------+--------+-----+----+---------------+
|   Felix|   49|  Cat|     2018-11-12|    null| null|null|           null|
|   Felix|   49|  Cat|     2018-11-12|    null| null|null|           null|
|Shinchan|   21|Human|     2018-06-05|    null| null|null|           null|
|Doraemon|   24|  Cat|     2019-01-02|Doraemon|   21| Cat|     2018-11-01|
|   Hachi|   68|  Bee|     2018-12-08|    null| null|null|           null|
+--------+-----+-----+---------------+--------+-----+----+---------------+



In [17]:
sdf1.join(sdf2, sdf1.name == sdf2.name, how='left').show()

+--------+-----+-----+---------------+--------+-----+----+---------------+
|    name|score| kind|enrollment_date|    name|score|kind|enrollment_date|
+--------+-----+-----+---------------+--------+-----+----+---------------+
|   Felix|   49|  Cat|     2018-11-12|    null| null|null|           null|
|   Felix|   49|  Cat|     2018-11-12|    null| null|null|           null|
|Shinchan|   21|Human|     2018-06-05|    null| null|null|           null|
|Doraemon|   24|  Cat|     2019-01-02|Doraemon|   21| Cat|     2018-11-01|
|   Hachi|   68|  Bee|     2018-12-08|    null| null|null|           null|
+--------+-----+-----+---------------+--------+-----+----+---------------+



# Drop duplicates <a id='drop_duplicates'></a>

### pandas

In [18]:
pdf1.drop_duplicates()

,name,score,kind,enrollment_date
0,Doraemon,24,Cat,2019-01-02
1,Hachi,68,Bee,2018-12-08
2,Shinchan,21,Human,2018-06-05
3,Felix,49,Cat,2018-11-12


###  SQL

In [19]:
sql_sc.sql("""    
select distinct *
from table1
""").show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Doraemon|   24|  Cat|     2019-01-02|
|Shinchan|   21|Human|     2018-06-05|
|   Felix|   49|  Cat|     2018-11-12|
|   Hachi|   68|  Bee|     2018-12-08|
+--------+-----+-----+---------------+



### spark

In [20]:
sdf1.distinct().show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Doraemon|   24|  Cat|     2019-01-02|
|Shinchan|   21|Human|     2018-06-05|
|   Felix|   49|  Cat|     2018-11-12|
|   Hachi|   68|  Bee|     2018-12-08|
+--------+-----+-----+---------------+



# Group By <a id='group_by'></a>

### pandas

#### count

In [21]:
pdf1['kind'].value_counts()

Cat      3
Human    1
Bee      1
Name: kind, dtype: int64

#### average

In [22]:
pdf1['score'].mean()

42.2

### sql 

#### count

In [23]:
sql_sc.sql("""    
select kind, count(*) as count_kind
from table1
group by kind
""").show()

+-----+----------+
| kind|count_kind|
+-----+----------+
|  Cat|         3|
|  Bee|         1|
|Human|         1|
+-----+----------+



#### Average

In [24]:
sql_sc.sql("""    
select avg(score) as score_avg
from table1
""").show()

+---------+
|score_avg|
+---------+
|     42.2|
+---------+



### spark

#### count

In [25]:
sdf1.groupBy('kind').count().show()

+-----+-----+
| kind|count|
+-----+-----+
|  Cat|    3|
|  Bee|    1|
|Human|    1|
+-----+-----+



In [26]:
sdf1.groupBy('kind').agg(count('kind').alias('count_kind')).show()

+-----+----------+
| kind|count_kind|
+-----+----------+
|  Cat|         3|
|  Bee|         1|
|Human|         1|
+-----+----------+



#### average

In [27]:
sdf1.select(mean(col('score')).alias('score_avg')).show()

+---------+
|score_avg|
+---------+
|     42.2|
+---------+



# Order by <a id='order_by'></a>

### pandas

In [28]:
pdf1.sort_values(by='score')

,name,score,kind,enrollment_date
2,Shinchan,21,Human,2018-06-05
0,Doraemon,24,Cat,2019-01-02
3,Felix,49,Cat,2018-11-12
4,Felix,49,Cat,2018-11-12
1,Hachi,68,Bee,2018-12-08


### sql

In [29]:
sql_sc.sql("""
select *
from table1
order by score
""").show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Shinchan|   21|Human|     2018-06-05|
|Doraemon|   24|  Cat|     2019-01-02|
|   Felix|   49|  Cat|     2018-11-12|
|   Felix|   49|  Cat|     2018-11-12|
|   Hachi|   68|  Bee|     2018-12-08|
+--------+-----+-----+---------------+



### spark

In [30]:
sdf1.orderBy('score').show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Shinchan|   21|Human|     2018-06-05|
|Doraemon|   24|  Cat|     2019-01-02|
|   Felix|   49|  Cat|     2018-11-12|
|   Felix|   49|  Cat|     2018-11-12|
|   Hachi|   68|  Bee|     2018-12-08|
+--------+-----+-----+---------------+



# Filter using where <a id='filter_using_where'></a>

### pandas

In [31]:
pdf1[pdf1['score']<30]

,name,score,kind,enrollment_date
0,Doraemon,24,Cat,2019-01-02
2,Shinchan,21,Human,2018-06-05


### sql

In [32]:
sql_sc.sql("""
select *
from table1
where score < 30
""").show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Doraemon|   24|  Cat|     2019-01-02|
|Shinchan|   21|Human|     2018-06-05|
+--------+-----+-----+---------------+



### spark

In [33]:
sdf1.where(sdf1.score < 30).show()

+--------+-----+-----+---------------+
|    name|score| kind|enrollment_date|
+--------+-----+-----+---------------+
|Doraemon|   24|  Cat|     2019-01-02|
|Shinchan|   21|Human|     2018-06-05|
+--------+-----+-----+---------------+



# Filter using like <a id='filter_using_like'></a>

### pandas

In [34]:
pdf1[pdf1['name'].str.contains("dora",case=False)]

,name,score,kind,enrollment_date
0,Doraemon,24,Cat,2019-01-02


### sql

In [35]:
sql_sc.sql("""
select *
from table1
where lower(name) like 'dora%'
""").show()

+--------+-----+----+---------------+
|    name|score|kind|enrollment_date|
+--------+-----+----+---------------+
|Doraemon|   24| Cat|     2019-01-02|
+--------+-----+----+---------------+



### spark

In [36]:
sdf1.where(lower(col('name')).like("dora%")).show()

+--------+-----+----+---------------+
|    name|score|kind|enrollment_date|
+--------+-----+----+---------------+
|Doraemon|   24| Cat|     2019-01-02|
+--------+-----+----+---------------+



# Count distinct values in a column <a id='count_distinct_values_in_a_column'></a>

### pandas

In [37]:
pdf1['kind'].unique()

array(['Cat', 'Bee', 'Human'], dtype=object)

In [38]:
pdf1['kind'].nunique()

3

### sql

In [39]:
sql_sc.sql("""
select distinct kind
from table1
""").show()

+-----+
| kind|
+-----+
|  Cat|
|  Bee|
|Human|
+-----+



In [40]:
sql_sc.sql("""
select count(distinct kind) as count_unique_kind
from table1
""").show()

+-----------------+
|count_unique_kind|
+-----------------+
|                3|
+-----------------+



### spark

In [41]:
sdf1.select("kind").distinct().show()

+-----+
| kind|
+-----+
|  Cat|
|  Bee|
|Human|
+-----+



In [42]:
sdf1.select(countDistinct("kind").alias('count_unique_kind')).alias('count_kind').show()

+-----------------+
|count_unique_kind|
+-----------------+
|                3|
+-----------------+



# Count null rows per column <a id='count_null_rows_per_column'></a>

### pandas

In [43]:
pdf1.describe()

,score
count,5.000000
mean,42.200000
std,19.613771
min,21.000000
25%,24.000000
50%,49.000000
75%,49.000000
max,68.000000


In [44]:
pdf1.isnull().sum()

name               0
score              0
kind               0
enrollment_date    0
dtype: int64

### sql

In [45]:
sql_sc.sql("""
describe table1
""").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|           name|   string|   null|
|          score|   bigint|   null|
|           kind|   string|   null|
|enrollment_date|   string|   null|
+---------------+---------+-------+



In [46]:
sql_sc.sql("""
select
sum(case when name is null then 1 else 0 end) as count_null_name,
sum(case when score is null then 1 else 0 end) as count_null_score,
sum(case when kind is null then 1 else 0 end) as count_null_kind
from table1
""").show()

+---------------+----------------+---------------+
|count_null_name|count_null_score|count_null_kind|
+---------------+----------------+---------------+
|              0|               0|              0|
+---------------+----------------+---------------+



### spark

In [47]:
sdf1.describe().show()

+-------+--------+------------------+-----+---------------+
|summary|    name|             score| kind|enrollment_date|
+-------+--------+------------------+-----+---------------+
|  count|       5|                 5|    5|              5|
|   mean|    null|              42.2| null|           null|
| stddev|    null|19.613770672667712| null|           null|
|    min|Doraemon|                21|  Bee|     2018-06-05|
|    max|Shinchan|                68|Human|     2019-01-02|
+-------+--------+------------------+-----+---------------+



In [48]:
sdf1.select([count(when(isnan(c), c)).alias('count_null_'+c) for c in sdf1.columns]).show()

+---------------+----------------+---------------+--------------------------+
|count_null_name|count_null_score|count_null_kind|count_null_enrollment_date|
+---------------+----------------+---------------+--------------------------+
|              0|               0|              0|                         0|
+---------------+----------------+---------------+--------------------------+



# Get items of a column in table A not present in a column of table B <a id='get_items_A_not_in_B'></a>

### pandas

In [49]:
pdf1[~pdf1['kind'].isin(pdf2['kind'])]['kind']

1    Bee
Name: kind, dtype: object

### sql

In [50]:
sql_sc.sql("""
select table1.kind
from table1
left anti join table2 on table2.kind = table1.kind
""").show()

+----+
|kind|
+----+
| Bee|
+----+



### spark

In [51]:
sdf1.select("kind")\
    .join(sdf2,['kind'],how='left_anti')\
    .show()

+----+
|kind|
+----+
| Bee|
+----+

